<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/061_Normalization_Layers_BN_vs_LN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛁 Normalization Layers: Batch vs Layer vs Instance

Bez normalizacji głębokie sieci nie działają (gradienty wybuchają lub znikają).
Musimy sprowadzić dane do wspólnego mianownika (średnia ~0, odchylenie ~1).

Ale jak to policzyć, gdy mamy tensor 4D? `[Batch, Channel, Height, Width]`.

1.  **Batch Norm (BN):** Patrzy na **ten sam kanał** we wszystkich zdjęciach w batchu.
    *   *"Jaki jest średni kolor czerwony w całym batchu?"*
    *   *Król CNN.* Wymaga dużego Batch Size.
2.  **Layer Norm (LN):** Patrzy na **wszystkie kanały** w jednym zdjęciu.
    *   *"Jaki jest średni sygnał w tym konkretnym zdaniu?"*
    *   *Król NLP (Transformerów).* Działa nawet przy Batch Size = 1.
3.  **Instance Norm (IN):** Patrzy na **jeden kanał w jednym zdjęciu**.
    *   *Król Style Transfer (GAN).* Ignoruje kontrast zdjęcia.

Zrobimy eksperyment na małym tensorze, żeby zobaczyć różnicę w liczbach.

In [1]:
import torch
import torch.nn as nn

# 1. PRZYGOTOWANIE DANYCH
# Tensor [Batch=2, Channels=3, H=2, W=2]
# Wyobraź sobie 2 malutkie obrazki 2x2 piksele RGB.

# Obrazek 1: Ciemny (same małe liczby: 0, 1, 2)
img1 = torch.tensor([
    [[0., 1.], [0., 1.]], # R
    [[2., 3.], [2., 3.]], # G
    [[0., 0.], [0., 0.]]  # B
])

# Obrazek 2: Jasny (same duże liczby: 10, 11, 12)
img2 = torch.tensor([
    [[10., 11.], [10., 11.]], # R
    [[12., 13.], [12., 13.]], # G
    [[10., 10.], [10., 10.]]  # B
])

# Sklejamy w Batch
batch = torch.stack([img1, img2])

print(f"Kształt: {batch.shape} (N, C, H, W)")
print("Obrazek 1: Małe wartości (średnia ~1.5)")
print("Obrazek 2: Duże wartości (średnia ~11.5)")

Kształt: torch.Size([2, 3, 2, 2]) (N, C, H, W)
Obrazek 1: Małe wartości (średnia ~1.5)
Obrazek 2: Duże wartości (średnia ~11.5)


## 1. Batch Normalization (Pionowo)

BN normalizuje "wzdłuż Batcha".
Dla Kanału R (Czerwonego) weźmie piksele z Obrazka 1 **ORAZ** z Obrazka 2.

*   Średnia Kanału R = (Średnia img1 + Średnia img2) / 2
*   Efekt: Obrazek 1 zostanie "podciągnięty" w górę, a Obrazek 2 "ściągnięty" w dół. **Tracimy informację, że jeden był ciemny, a drugi jasny!** Ale zyskujemy stabilność cech.

In [2]:
# Inicjalizacja BatchNorm (num_features = liczba kanałów = 3)
bn = nn.BatchNorm2d(num_features=3)

# Przepuszczamy dane (w trybie treningowym, żeby liczył statystyki)
bn.train()
out_bn = bn(batch)

print("--- BATCH NORM ---")
print("Obrazek 1 (Ciemny):\n", out_bn[0, 0, :, :]) # Kanał R
print("Obrazek 2 (Jasny):\n", out_bn[1, 0, :, :]) # Kanał R

print("\nWNIOSEK:")
print("Wartości w obu obrazkach są teraz podobne (ok. -1 i +1).")
print("BN wymieszał statystyki obu obrazków.")

--- BATCH NORM ---
Obrazek 1 (Ciemny):
 tensor([[-1.0945, -0.8955],
        [-1.0945, -0.8955]], grad_fn=<SelectBackward0>)
Obrazek 2 (Jasny):
 tensor([[0.8955, 1.0945],
        [0.8955, 1.0945]], grad_fn=<SelectBackward0>)

WNIOSEK:
Wartości w obu obrazkach są teraz podobne (ok. -1 i +1).
BN wymieszał statystyki obu obrazków.


## 2. Layer Normalization (Poziomo)

LN normalizuje "wewnątrz Obrazka".
Dla Obrazka 1 weźmie wszystkie jego piksele (R, G, B) i policzy ich średnią.
Nie obchodzi go Obrazek 2.

*   Efekt: Zachowuje niezależność próbek. Dlatego jest kluczowy w NLP, gdzie jedno zdanie może być długie, a drugie krótkie, i nie chcemy, żeby jedno wpływało na drugie.

In [3]:
# Inicjalizacja LayerNorm
# Musimy podać kształt tego, co normalizujemy (C, H, W)
ln = nn.LayerNorm([3, 2, 2])

out_ln = ln(batch)

print("--- LAYER NORM ---")
print("Obrazek 1:\n", out_ln[0, 0, :, :])
print("Obrazek 2:\n", out_ln[1, 0, :, :])

print("\nWNIOSEK:")
print("Każdy obrazek został znormalizowany OSOBNO.")

--- LAYER NORM ---
Obrazek 1:
 tensor([[-0.8660,  0.0000],
        [-0.8660,  0.0000]], grad_fn=<SelectBackward0>)
Obrazek 2:
 tensor([[-0.8660,  0.0000],
        [-0.8660,  0.0000]], grad_fn=<SelectBackward0>)

WNIOSEK:
Każdy obrazek został znormalizowany OSOBNO.


## 3. Instance Normalization (Styl)

IN idzie o krok dalej. Normalizuje każdy kanał w każdym obrazku osobno.
*   Obrazek 1, Kanał R -> Osobna średnia.
*   Obrazek 1, Kanał G -> Osobna średnia.

**Dlaczego to ważne?**
To usuwa "styl" (kontrast, jasność globalną) z konkretnego koloru. Dlatego jest używane w **Style Transfer** (np. przerabianie zdjęcia na obraz Van Gogha). Chcemy zachować kształty (treść), ale podmienić statystykę kolorów (styl).

In [4]:
# Inicjalizacja InstanceNorm
in_layer = nn.InstanceNorm2d(num_features=3)

out_in = in_layer(batch)

print("--- INSTANCE NORM ---")
print("Wynik:\n", out_in[0, 0, :, :])

--- INSTANCE NORM ---
Wynik:
 tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]])


## 🧠 Podsumowanie: Ściąga Architekta

Kiedy czego używać?

1.  **BatchNorm:**
    *   **Gdzie:** CNN, Computer Vision (ResNet, YOLO).
    *   **Warunek:** Musisz mieć duży Batch Size (>32). Przy małym Batchu (np. 2) średnia jest zaszumiona i BN psuje trening.
    
2.  **LayerNorm:**
    *   **Gdzie:** NLP, Transformery (BERT, GPT), RNN.
    *   **Zaleta:** Działa niezależnie od Batch Size. Idealne, gdy dane wejściowe mają różną długość.

3.  **InstanceNorm:**
    *   **Gdzie:** Generative AI (GAN, Style Transfer).
    *   **Cel:** Gdy chcesz manipulować stylem obrazu, a nie jego treścią.

4.  **GroupNorm:** (Hybryda)
    *   Dzieli kanały na grupy. Używane w detekcji obiektów, gdy Batch Size musi być mały (bo obrazki są wielkie 4K).